In [1]:
%matplotlib inline

In [13]:
%run dog-breed-cnn.py
# Unzip the train and test zip file
archive_train = ZipFile("Data/test.zip", "r")
archive_test = ZipFile("Data/train.zip", "r")

# This line shows the 5 first image name of the train database
archive_train.namelist()[0:5]

# This line shows the number of images in the train database, 
# noted that we must remove the 1st value (column header)
len(archive_train.namelist()[:]) - 1

10357

In [14]:
%run dog-breed-cnn.py

image_resize = 60

DataBase_creator(archivezip=archive_train, nwidth=image_resize, nheight=image_resize, save_name="train")

DataBase_creator(archivezip=archive_test, nwidth=image_resize, nheight=image_resize, save_name="test")


Time elapsed: 0:02:15
Time elapsed: 0:01:47


In [18]:
# Load TRAIN
train = pickle.load(open("train.p","rb"))
train.shape

(10357, 60, 60, 3)

In [17]:
# Load TEST
test = pickle.load(open("test.p","rb"))
test.shape

(10222, 60, 60, 3)